In [1]:
import keras
keras.__version__

Using TensorFlow backend.
/home/nicholas/Deep_learn_in_Py/deep-learning-with-python-notebooks-master/deep_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nicholas/Deep_learn_in_Py/deep-learning-with-python-notebooks-master/deep_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nicholas/Deep_learn_in_Py/deep-learning-with-python-notebooks-master/deep_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a f

'2.0.8'

# Text generation with LSTM

This notebook contains the code samples found in Chapter 8, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

[...]

## Implementing character-level LSTM text generation


Let's put these ideas in practice in a Keras implementation. The first thing we need is a lot of text data that we can use to learn a 
language model. You could use any sufficiently large text file or set of text files -- Wikipedia, the Lord of the Rings, etc. In this 
example we will use some of the writings of Nietzsche, the late-19th century German philosopher (translated to English). The language model 
we will learn will thus be specifically a model of Nietzsche's writing style and topics of choice, rather than a more generic model of the 
English language.

## Preparing the data

Let's start by downloading the corpus and converting it to lowercase:

In [2]:
import keras
import numpy as np

#path = keras.utils.get_file(
#    'nietzsche.txt',
#    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

#text = open(path).read().lower()

path ='/home/nicholas/Downloads/newTest.txt'
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 998506



Next, we will extract partially-overlapping sequences of length `maxlen`, one-hot encode them and pack them in a 3D Numpy array `x` of 
shape `(sequences, maxlen, unique_characters)`. Simultaneously, we prepare a array `y` containing the corresponding targets: the one-hot 
encoded characters that come right after each extracted sequence.

In [3]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 332816
Unique characters: 48
Vectorization...


## Building the network

Our network is a single `LSTM` layer followed by a `Dense` classifier and softmax over all possible characters. But let us note that 
recurrent neural networks are not the only way to do sequence data generation; 1D convnets also have proven extremely successful at it in 
recent times.

In [4]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Since our targets are one-hot encoded, we will use `categorical_crossentropy` as the loss to train the model:

In [5]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Training the language model and sampling from it


Given a trained model and a seed text snippet, we generate new text by repeatedly:

* 1) Drawing from the model a probability distribution over the next character given the text available so far
* 2) Reweighting the distribution to a certain "temperature"
* 3) Sampling the next character at random according to the reweighted distribution
* 4) Adding the new character at the end of the available text

This is the code we use to reweight the original probability distribution coming out of the model, 
and draw a character index from it (the "sampling function"):

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


Finally, this is the loop where we repeatedly train and generated text. We start generating text using a range of different temperatures 
after every epoch. This allows us to see how the generated text evolves as the model starts converging, as well as the impact of 
temperature in the sampling strategy.

In [7]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
332816/332816 [==============================] - 248s - loss: 1.6709   
--- Generating with seed: "ad been the face of an angel.

7:1 then said the high priest"
------ temperature: 0.2
ad been the face of an angel.

7:1 then said the high priest of therefore said, and said, and said, and said, and therefore said, and said, and therefore therefore therefore therefore therefore shall be be part therefore therefore therefore therefore therefore therefore therefore and therefore and therefore therefore therefore therefore shall be not and therefore therefore therefore therefore and therefore and therefore therefore therefore therefore theref
------ temperature: 0.5
therefore and therefore therefore therefore therefore therefore proffer of there aran of the father.

1:2 and he had spoken of to men and unto there shall be part therefore therefore therefore therefore therefore and
ben and
therefore therefore therefore alto among therefore that he shall not into his saw, and 

we even pirate of to thou, one of jerusalem, fordvenuchch.

7:15 and so
with framble conssueth trught desprice no among umought on him, and he hope much
pilate have known, and is godved are judgmen, lutiin
se tibareth, and wort ; behold the own brayet.

17:16 out for th; there so matisob, and will given nothing to
epoch 5
Epoch 1/1
332816/332816 [==============================] - 241s - loss: 1.2334   
--- Generating with seed: ", then cometh the wicked one, and catcheth away that which w"
------ temperature: 0.2
, then cometh the wicked one, and catcheth away that which were
called the same there was the son of god.

12:16 and he said, which have not depart in the son of the sea, and
there are the season, and all the son of god.

14:21 then said they had dead unto them, and said, and come that we
do not there was the sealed unto them, and they had dead.

12:11 therefore they had dead also and sins, and the season, and
they had dead and said, and said, they had spe
------ temperature: 

they tempentless on you generation, and all domandel is the son of
deceiving on his secon.

11:18 and are mary them out contralyire of 
------ temperature: 1.2
g on his secon.

11:18 and are mary them out contralyire of all the tartain.

5:24and when he had thou is jesus a garsect.

11:12 but when he hadspean forth her thou furs memniss proceerncaed
take us by maither of many, beare from is, and harpen also revarned, for
to yourselves them, he asked themnings no heaf; now herients;
1:1asure spake, which was deceuded to et? we justestfsurpwirmosel ; and
brethren, and brother and deceivion, became of me.

3:4 it is
epoch 9
Epoch 1/1
332816/332816 [==============================] - 240s - loss: 1.1924   
--- Generating with seed: "came unto me, and
stood, and said unto me, brother saul, rec"
------ temperature: 0.2
came unto me, and
stood, and said unto me, brother saul, receive the lord and the
son of god, and the servants of the chief priests and the son of god.

1:19 and the son of g

/home/nicholas/Deep_learn_in_Py/deep-learning-with-python-notebooks-master/deep_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


od, and the lord the sea, and the son of god.

1:14 and the son of god, and the word of the brethren and the son of god.

1:1 and when he was all the son of god, and the son of god, and
the son of god, and the world of the son of god, and the lord and the councely i say unto you,
and t
------ temperature: 0.5
 of god, and the lord and the councely i say unto you,
and the same in the lord the heart they said about the house.

1:14 and she shall be to be come and said, not all the part of the
word, and shall be not the son of god.

12:15 and the body of the son of god, brethren through the caused, and of the man, and or every that
and an every the house, and when the third spirit do the kingdom.

4:19 and the four lord good prophecy, which were the heart and when
------ temperature: 1.0
d the four lord good prophecy, which were the heart and when
faif to are rued upon inforth with this spirit.

17:3 for this fore, all man taken by elirmort worthy of the world.

23:4 now an uphime if that

332816/332816 [==============================] - 241s - loss: 1.1713   
--- Generating with seed: "emselves swift destruction.

2:2 and many shall follow their"
------ temperature: 0.2
emselves swift destruction.

2:2 and many shall follow their hand of the prophets of the body
of the chief priests and the father shall be before the beast and the
spirit the spirit of the beginning.

15:14 and he said unto them, i say unto you, the spirit the make the
will of god and said, i say unto you, the spirit the spirit of
the word of the word of the word of the same said, it were
the prophets and the beginning.

10:1 and he said unto him, like the
------ temperature: 0.5
hets and the beginning.

10:1 and he said unto him, like the father.

17:10 saying, except this day the depart to be found the lord.

15:38 then his disciples said unto him, being set dead, and the angels
of the husband, whose days of god shall be condemnation of us
not preached about upon the word with the people in the father 

hath tongues, buid jesus, the fruit, even as ought of usresling isrut
perils.

22:5 and she do thet thence till it state, whom also he was contenus?  10:9 but my
father unto thiem of righteousnesstdploes rendycleath, which have
body, that try affilelly i will relason, yunder grees heno? and
while rose were hath doy him that sp
epoch 17
Epoch 1/1
332816/332816 [==============================] - 241s - loss: 1.1716   
--- Generating with seed: " in him: 1:11 in
whom also we have obtained an inheritance, "
------ temperature: 0.2
 in him: 1:11 in
whom also we have obtained an inheritance, and the grace of the
works of the house of the spirit of the same standing of the
father of the spirit of the way of the saints, and said unto
them, and the spirit of the spirit of the same saints.

14:10 and the man were the land of the law, and he that shall be
come one of the spirit of the same saith, and the same said unto
them, and the son of the way of the same prophets, and the sea
and all 
------

11:11 accord impasany thy, even fire until day of thing other by his
see, say to you, he shall make do much angel of
the: and the neye of thy doors.

4:21 for i ev
------ temperature: 1.2
much angel of
the: and the neye of thy doors.

4:21 for i ever: 13:12 he raiaing to then.

10:14 saive a woman, and thepplise, which shall be a more shed doin: 5:nas, we givone see that
she came, rul as out of the; 1:6 forsi firso.

6:9 r his devilcredity begat his heart, for them, ors thing of thy
disgives to precaye me from these such a echy felixf, and he
need and agod: for ye ever bhing.

2:4 dsives man chould no more blfor: 7:8 whosoever came rebuk fo
epoch 21
Epoch 1/1
171392/332816 [==============>...............] - ETA: 117s - loss: 1.1754

KeyboardInterrupt: 


As you can see, a low temperature results in extremely repetitive and predictable text, but where local structure is highly realistic: in 
particular, all words (a word being a local pattern of characters) are real English words. With higher temperatures, the generated text 
becomes more interesting, surprising, even creative; it may sometimes invent completely new words that sound somewhat plausible (such as 
"eterned" or "troveration"). With a high temperature, the local structure starts breaking down and most words look like semi-random strings 
of characters. Without a doubt, here 0.5 is the most interesting temperature for text generation in this specific setup. Always experiment 
with multiple sampling strategies! A clever balance between learned structure and randomness is what makes generation interesting.

Note that by training a bigger model, longer, on more data, you can achieve generated samples that will look much more coherent and 
realistic than ours. But of course, don't expect to ever generate any meaningful text, other than by random chance: all we are doing is 
sampling data from a statistical model of which characters come after which characters. Language is a communication channel, and there is 
a distinction between what communications are about, and the statistical structure of the messages in which communications are encoded. To 
evidence this distinction, here is a thought experiment: what if human language did a better job at compressing communications, much like 
our computers do with most of our digital communications? Then language would be no less meaningful, yet it would lack any intrinsic 
statistical structure, thus making it impossible to learn a language model like we just did.


## Take aways

* We can generate discrete sequence data by training a model to predict the next tokens(s) given previous tokens.
* In the case of text, such a model is called a "language model" and could be based on either words or characters.
* Sampling the next token requires balance between adhering to what the model judges likely, and introducing randomness.
* One way to handle this is the notion of _softmax temperature_. Always experiment with different temperatures to find the "right" one.